# Quick Start - pyCityLayers

# Access to Internal (CERC) Data

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import json
from pprint import pprint
import fiona
from shapely.geometry import Polygon, Point, MultiPolygon
from pycitylayers.client import Client
from pycitylayers.utils import PointGQL, PolygonGQL


### Create an instance of connection to database

In [ ]:
client = Client().create(source='cerc')

### Querying for all collections available in the database

In [ ]:
coll = client.collection
pprint(coll)

### Show all datasets in the collection

In [ ]:
coll.datasets

In [ ]:
ds = coll[2]
ds

### Show all tables in a dataset

In [ ]:
ds.tables

### Show all columns in a table

In [ ]:
tb = ds[0]
columns = tb.columns
pprint(columns)

### Get N rows from a table given a list of columns 

In [ ]:
query_options = {
    'columns': ['code', 'geom', 'index', 'name', 'year'], 
    'nrows': 500, 
    'skiprows': 0,
}

data = tb.query_simple( columns=['code', 'geom', 'index', 'name', 'year'], nrows=500 )
# data = tb.query_simple( **query_options )

print("Table Size: ", data.shape)
data


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

list_of_points = [ Point(row['geom']['coordinates']) for _, row in data.iterrows() ]

gdf = gpd.GeoDataFrame(index=list(range(len(list_of_points))), crs='epsg:4326', geometry=list_of_points)

# load neighbourhood shape file
mtl_neighs = gpd.read_file('montreal_neighs.geojson')


fig, ax = plt.subplots(figsize=(12, 10))
mtl_neighs.plot(ax=ax, color=None, alpha=0.69, facecolor='white', edgecolor='black', linewidth=1)
gdf.plot(ax=ax, color='r', alpha=0.2, facecolor='red', edgecolor=None, linewidth=0)

ax.set_xlim(-74.05, -73.4)
ax.set_ylim(45.35, 45.75)



### Get N rows from a table given a list of columns and distance to a point

In [ ]:
query_geometry = PointGQL().point(-73.64453315734863,45.46627091868821)
print( query_geometry )


query_options = {
    'columns': ['code', 'name', 'year'], 
    'nrows': 500, 
    'skiprows': 0,
    'geometry_operation': 'distance_from_point',
    'geom_distance': 0.015,
    'geometry': query_geometry,
    'crs_epsg': 4326,
}

data = tb.query_simple( **query_options )

print("Table Size: ", data.shape)
data


In [ ]:
ds = coll.search(by_name="lots")[0]
tb = ds[0]
tb

In [ ]:
query_geometry = PointGQL().point(-73.64453315734863,45.46627091868821)
print( query_geometry )


query_options = {
    'columns': ['index', 'built_area', 'geom', 'year_built'], 
    'nrows': 10000, 
    'skiprows': 0,
    'geometry_operation': 'distance_from_point',
    'geom_distance': 0.005,
    'geometry': query_geometry,
    'crs_epsg': 4326,
}

data = tb.query_simple( **query_options )

print("Table Size: ", data.shape)
data


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

list_of_polygons = [Polygon([Point(el) for el in poly['geom']['coordinates'][0]]) for _,poly in data.iterrows()]

gdf = gpd.GeoDataFrame(index=list(range(len(data))), crs='epsg:4326', geometry=list_of_polygons)

gdf.plot(color='r', alpha=0.7, facecolor='white', edgecolor='black', linewidth=2, figsize=(12,12))


### Get N rows from a table given a list of columns and within a polygon

In [ ]:
query_geometry = PolygonGQL().rect_from_two_corners([-73.643610,45.468896], [-73.623610,45.458896])
print( query_geometry )


query_options = {
    'columns': ['index', 'built_area', 'geom', 'year_built'], 
    'nrows': 10000, 
    'skiprows': 0,
    'geometry_operation': 'is_within_poly',
    'geometry': query_geometry,
    'crs_epsg': 4326,
}

data = tb.query_simple( **query_options )

print("Table Size: ", data.shape)
data

## visualize results

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

def rgb(minimum, maximum, value):
    if value < minimum: value = minimum
    if value > maximum: value = maximum
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return [r/255, g/255, b/255]

list_of_polygons = [Polygon([Point(el) for el in poly['geom']['coordinates'][0]]) for _, poly in data.iterrows()]

gdf = gpd.GeoDataFrame(index=list(range(len(data))), crs='epsg:4326', geometry=list_of_polygons)
colors = data['year_built'].apply(lambda x: rgb(1900,2020,x))
gdf.plot(color=colors, alpha=0.7, facecolor='white', edgecolor='black', linewidth=2, figsize=(12,12))



## Save as file

In [ ]:
from pathlib import Path  
filepath = Path('folder/subfolder/out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)